In [78]:
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from PIL import Image
import os
from pathlib import Path
from utils import *
from allResNets import *
from engine import *
from data_transforms import create_transform
from dataloaders import *
import cv2
from data_preparation import *
import torch
import glob
import pandas as pd

In [79]:
def find_classes(csv_file: str) -> List[str]:
    classes = []
    
    df = pd.read_csv(csv_file)

    for index, row in df.iterrows():
        labels = []

        if(row['Camera Alignment']):
            labels.append('Camera Alignment')

        if(row['Obstructed Camera']):
            labels.append('Obstructed Camera')

        if(row['Over lighting']):
            labels.append('Over lighting')

        if(row['Under lighting']):
            labels.append('Under lighting')

        if(row['Saturated']):
            labels.append('Saturated')

        classes.append(labels)
    
    return classes
    


In [86]:
class CustomImageFolderMultiLabel(torch.utils.data.Dataset):
    def __init__(self, csv_file, transform = None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.image_paths = "Quality/dataset/" + self.data.iloc[:,-1] + "/" +  self.data.iloc[:,0] + ".png"
        self.labels = self.data.iloc[:,2:7]
        self.classes = find_classes(csv_file)

    def __getitem__(self, index):
        image_path = self.image_paths.iloc[index]
        label = self.labels.iloc[index]
        classes = self.classes[index]
        image = Image.open(image_path).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label, classes
    
    def __len__(self):
        return len(self.image_paths)

In [87]:
image_shape = (1080, 1920)
resize_factor = 0.3
resize = np.multiply(image_shape,resize_factor)
resize = [int(resize[0]), int(resize[1])]
BATCH_SIZE = 16

data_transform = create_transform(resize=resize, rotate=10, flip_h = True, color_dev=True, transf_tensor=True, normalize=True, sp_noise=True, gauss_noise=True)

data = CustomImageFolderMultiLabel(csv_file="Quality/all_data_labeled.csv",transform=data_transform)

In [89]:
image, label, classes = data.__getitem__(0)
image, label, classes

(tensor([[[-0.8510, -0.8510, -0.8431,  ..., -0.8510, -0.8431, -0.8510],
          [-0.8510, -0.8431, -0.8431,  ..., -0.8510, -0.8431, -0.8510],
          [-0.8510, -0.8510, -0.8510,  ..., -0.8510, -0.8510, -0.8431],
          ...,
          [-0.8431, -0.8431, -0.8510,  ..., -0.8510, -0.8431, -0.8431],
          [-0.8510,  0.9922, -0.8431,  ..., -0.8431, -0.8431, -0.8510],
          [-0.8510, -0.8431, -0.8431,  ..., -0.8510, -0.8431, -0.8431]],
 
         [[-0.8510, -0.8431, -0.8431,  ..., -0.8431, -0.8431, -0.8510],
          [-0.8510, -0.8510, -0.8510,  ..., -0.8431, -0.8510, -0.8510],
          [-0.8431, -0.8510, -0.8510,  ..., -0.8431, -0.8510, -0.8510],
          ...,
          [-0.8510, -0.8510, -0.8431,  ..., -0.8510, -0.8431, -0.8510],
          [-0.8510,  1.0000, -0.8510,  ..., -0.8510, -0.8510, -0.8510],
          [-0.8510, -0.8510, -0.8431,  ..., -0.8510, -0.8431, -0.8431]],
 
         [[-0.8431, -0.8431, -0.8431,  ..., -0.8510, -0.8510, -0.8431],
          [-0.8431, -0.8510,